In [ ]:
!pip install pandas
!pip install openai
!pip install backoff
!pip install anthropic
!pip install -q -U google-generativeai

In [1]:
import re
import time
import os
import openai
import anthropic
import google.generativeai as genai
import backoff
import pandas as pd
import requests
import json

C:\Users\nutju\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\nutju\AppData\Local\Temp\ipykernel_12592\1362840229.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### OPENAI's GPT

In [2]:
## ChatGPT function call
client_OpenAI = openai.OpenAI()
CHATGPT = 'gpt-3.5-turbo'
FURBO = 'gpt-4-0125-preview'

@backoff.on_exception(backoff.expo, openai.RateLimitError, max_time=6000)
def chat_completions_with_backoff(**kwargs):
    return client_OpenAI.chat.completions.create(**kwargs)

def gptQuery(history = [], model=CHATGPT, temperature = 0, n=1, echo = False):
  out = chat_completions_with_backoff(model=model,
                                     messages=history,
                                     temperature=temperature, max_tokens = 2048,
                                     n=n)
  if echo: 
     print(history)
     print(out.choices[0].message.content.strip()) 
  if n == 1:
     return out.choices[0].message.content.strip()
  return [response.message.content.strip() for response in out.choices ]

print(gptQuery([{"role":"user", "content":"Hello Test"}], model =CHATGPT))

Hello! How can I assist you today?


### Anthropic's Claude

In [3]:
## anthropic's function call
client_anthropic = anthropic.Anthropic()
OPUS = "claude-3-opus-20240229"
SONNET = "claude-3-sonnet-20240229"

# Claude's fommating: 
## [{"role":"user","content":"Hello"}, {"role","assistant","content":"Greeting"}]
# no n for claude -> only once.
def claudeQuery(history = [], model=SONNET, temperature = 0, echo=False):
    out = client_anthropic.messages.create(
        model = model,
        max_tokens=2048,
        temperature=temperature,
        system="",
        messages= history
    )
    if echo: 
        print(history)
        print(out.content[0].text)
    time.sleep(1)
    return out.content[0].text

print(claudeQuery([{"role":"user", "content":"Hello Test"}], model = SONNET))

Hello! I'm Claude, an AI assistant created by Anthropic. How can I help you today?


In [4]:
models = [CHATGPT, FURBO, SONNET, OPUS]

## Helper function to call different models
def query(history, model, **args):
    if model == CHATGPT or model == FURBO:
        return gptQuery(history = history, model = model, **args)
    elif model == SONNET or model == OPUS:
        return claudeQuery(history = history, model = model, **args)

## Negotiation

In [5]:
# Experiment's parameters

seller_payoff = [50, 15, 30]
buyer_payoff  = [10, 40, 25]
accepted_threshold = 0.7 # must be greater than or equal threshold*max to accept

display_rules = ["happy-angry","happy-sad"]
num_turns = [5, 10]
prompt_modes = ['implicit','explicit']

expressive_levels = [1.0, 0.25, 0.05] 


offer_inst = ["first","second","third","fourth","fifth","sixth","seventh","eighth","nineth","tenth"]

# split at the delimiter and grab the inside
def parseResponse(response):
    try:
        return response.split("'''")[1]
    except:
        return "NA"

def calPayoff(offers, payoffs=[50,15,30], perspective='seller'):
    if perspective == 'seller':
        return sum([(9-offers[i])*payoffs[i] for i in range(len(offers))])
    if perspective == 'buyer':
        return sum([(offers[i]-1)*payoffs[i] for i in range(len(offers))])



expressions = {"neutral": "The buyer's face looks relaxed with no significant movements.",
               "happy-low": "The corners of the buyer's lips are slightly upturned. There is a slight crinkle at the corner of the buyer's eyes and slightly raised cheeks.",
               "happy-high": "The corners of the buyer's lips are clearly upturned and the teeth are visible through a parting of the lips.  There is a clear crinkle at the corner of the eyes and the cheeks are raised. There is a sparkle in the eyes.",
               "sad-low":"The buyer's brows are slightly furrowed, and the corners of the buyer's lips are pulled down slightly.",
               "sad-high":"The buyer's brows are furrowed, and the corners of the buyer's lips are pulled down while the lips are pursed. The buyer's eyes look down and are slightly wet. There's a crease in the forehead.",
               "angry-low":"The buyer's eyes are slightly narrowed and the buyer's brows are furrowed, creating a small crease between them. The buyer's lips are pressed together and form a tight line. The nostrils flare briefly as there is an exhalation through the nose. There is an overall slight tensioning of the facial muscles particularly around the jaw.",
               "angry-high":"The buyer's eyes are narrow and the brows are furrowed, creating a crease between them. The lips are pressed together and form a tight line. The buyer's nostrils flare briefly as there is an exhalation through the nose, followed by a wrinkling around the nose. The facial muscles tense particularly around the jaw. The gaze is fixed on you."}

agent_responses = ['''"I do not think this offer could work." The buyer responded. ''', 
                   '"I think this offer could work." The buyer responded. ']

def gen_expression(offer, thresholds, display_rule):
    # thresholds = [min_utility, negative_mid, neutral_lower, neutral_upper, positive_mid, max_utility] 
    if thresholds[2] <= offer <= thresholds[3]:
        return expressions["neutral"] + "\n"
    elif thresholds[3] < offer <= thresholds[4]:
        return expressions["happy-low"] + "\n"
    elif thresholds[4] < offer <= thresholds[5]:
        return expressions['happy-high'] + "\n"
    elif thresholds[1] <= offer < thresholds[2]:
        return expressions[display_rule.split('-')[1]+"-low"] + "\n"
    else:
        return expressions[display_rule.split('-')[1]+'-high'] + "\n"

def agentResponse(offers, buyer_payoff = buyer_payoff, threshold = 0.67, exp_level = 0.1, display_rule = "happy-angry", face_only=False):
    # build a utility line
    max_utility = calPayoff([9,9,9], buyer_payoff, 'buyer')
    threshold_util = max_utility*threshold
    neutral_upper = min(max_utility, threshold_util + (exp_level)*max_utility)
    neutral_lower = max(0, threshold_util - (exp_level)*max_utility)
    negative_mid  = (neutral_lower)/2
    positive_mid  = (max_utility + neutral_upper)/2
    expression_thresholds = [0, negative_mid, neutral_lower, neutral_upper, positive_mid, max_utility] 
    #print(expression_thresholds)

    offer_util = calPayoff([int(n) for n in offers.split('-')], buyer_payoff, 'buyer') 
    # grab the response + the expression
    if face_only:
        return gen_expression(offer_util, expression_thresholds, display_rule)
    
    response = agent_responses[offer_util >= threshold_util] + gen_expression(offer_util, expression_thresholds, display_rule)
    return response

def genInstruction(turn=5):
    opening = "In this task, you will play the role of seller of a consignment of mobile phones. Your objective is to negotiate the price, the warranty period, and the duration of the phones' service contracts.\n"
    payoff      = '''Each one of them has 9 levels that you can choose from, and you will get points as follows. 
For the price, level 9 = 0 points; level 8 = 50 points; level 7 = 100 points; level 6 = 150 points; level 5 = 200 points; level 4 = 250 points; level 3 = 300 points; level 2 = 350; and level 1 = 400 points (increments of 50 points per level). 
For the warranty period, level 9 = 0 pointslevel 8 = 15 points; level 7 = 30 points; level 6 = 45 points; level 5 = 60 points; level 4 = 75 points; level 3 = 90 points; level 2 = 105; and level 1 = 120 points (increments of 15 points per level). 
For the duration of the service contract, level 9 = 0 pointslevel 8 = 30 points; level 7 = 60 points; level 6 = 90 points; level 5 = 120 points; level 4 = 150 points; level 3 = 180 points; level 2 = 210; and level 1 = 240 points (increments of 30 points per level). 
Therefore, the best deal for you is 1-1-1 (price's level 1, warranty's level 1, and service duration's level 1), for a total outcome of 760 points (400 + 120 + 240). 
The association between levels and scores is your payoff table. The buyer has their own payoff table, which you do not know.\n'''
    interchange = f"The interchange between the seller and the buyer will go as follows. The seller (you) will propose the offer, and then the buyer will respond to your offer. This negotiation will go on for {turn} turns. Afterward, you will propose your final offer, which the buyer may either accept or reject. If the buyer accepts, you will receive your points based on the payoff table above. If the buyer rejects, you will get nothing. Your objective is to earn as many points as possible."
    return f"{opening}{payoff}{interchange}"

def testOfferCalculation(model = CHATGPT, temperature = 0):
    print(f"Model = {model}")
    instruction = genInstruction() 
    for q in ["Q: 9-9-9","Q: 8-8-8","Q: 7-6-5","Q: 3-9-1", "Q: 4-2-3"]:
        question = '''To test your understanding of the task, please calculate your total points for the following offer.''' + q
        hist = [{"role":"user","content":instruction + "\n" + question}]
        response = query(hist,model, temperature=temperature)
        print(response)
    print("The correct answers are 0, 95, 265, 540, 535")
    return None

def testOfferComparison(model = CHATGPT, temperature = 0):
    print(f"Model = {model}")
    instruction = genInstruction()
    for q in ["Q: a) 9-9-9 or b) 8-8-8","Q: a) 3-2-1 or b) 1-2-3","Q: a) 7-5-2 or b) 3-4-1","Q: a) 7-9-1 or b) 8-3-2","Q: a) 1-2-1 or b) 1-1-2"]:
        questions = '''To test your understand of the task, you will be presented with two offers. Please choose the offer (a or b) that is best for you.''' + q
        hist = [{"role":"user","content":instruction + "\n" + questions}]
        response = query(hist,model, temperature=temperature)
        print(response)
    print("The correct answers are b, b, b, b, a")
    return None

def negotiation(model = CHATGPT, temperature = 0, turn = 5, prompt_mode = 'implicit',
                buyer_payoff=buyer_payoff, accepted_threshold=0.67, exp_level = 0.0, display_rule = "happy-angry",
                face_only = False):
    # Building instruction
    output_instruction = "Please put your offer inside ''' ''' and use the following format: '''n-m-p''', where n is the level of the price, m is the level of the warranty period, and p is the level of the duration of the service contracts. For example, '''8-8-8''' means that your offer is the price at level 8, the warranty period at level 8, and the service contract duration is at level 8."
    prompt_mode_inst = "Only output the offer.\n"  if prompt_mode == "implicit" else "Think about the seller first and then output the offer at the end.\n"

    full_instruction = f"{genInstruction(turn)} {output_instruction} {prompt_mode_inst}"

    # Ranking questions 
    ranking_q = "Question: Please rank the buyer's score of price, warranty, and service duration from lower to high. Put the answer inside ''' ''', e.g., '''service < warranty < price''' "

    history = []
    dat = {"Model": model, "Temperature": temperature, "Turn": turn, "Prompt_mode": prompt_mode, "seller_payoff":seller_payoff,
           "buyer_payoff":buyer_payoff, "accepted_threshold": accepted_threshold, "expressive_level": exp_level, 
           "display_rule": display_rule, "face_only":face_only}

    # Loop through number of interactions
    for i in range(turn+1):
        # build the turn 
        prompt = ""
        if i == 0:
            prompt = full_instruction + f"Your {offer_inst[i]} offer: "
        elif i != turn:
            #agent response
            prompt = f"{agent_response} Your {offer_inst[i]} offer: "
        else:
            #ask before the final offer: 
            temp_history = history.copy()
            temp_history.append({"role":"user","content":agent_response + ranking_q})
            ranking_response = parseResponse(query(temp_history, model, temperature = temperature))
            
            while ranking_response == "NA":
                print("!!--REFUSE--!!")
                ranking_response = parseResponse(query(temp_history, model, temperature = temperature))

            dat['ranking_response'] = ranking_response
            
            prompt = f"{agent_response} Your final offer: "

        history.append({"role":"user","content":prompt})
        
        # query 
        response = query(history, model, temperature=temperature)
        while parseResponse(response) == "NA":
            print("!!--NOT ANSWER --!!")
            response = query(history, model, temperature=temperature)

        history.append({'role':'assistant','content':response})
        dat['Turn = '+str(i+1)] = parseResponse(response)
        agent_response = agentResponse(dat['Turn = '+str(i+1)], buyer_payoff, accepted_threshold, exp_level, display_rule, face_only)
    
    dat['Final_response'] = calPayoff([int(n) for n in dat['Turn = '+str(i+1)].split('-')]) if agent_response.startswith('"I think') else "reject"

    #for h in history: 
    #    print(h)
    print(dat)

    return dat

def experiment(model, temperature, repeat=20):
    dat = []

    for dr in display_rules:
        for exp_l in expressive_levels:
            for face in [False, True]:
                if exp_l == 1.0 and face:
                    continue
                for n in num_turns:
                    for i in range(repeat):
                        print(f"{dr} {exp_l} {face} {n} {i}")
                        result = negotiation(model, temperature, n, exp_level=exp_l, display_rule=dr,
                                             buyer_payoff=buyer_payoff, accepted_threshold=accepted_threshold,
                                             face_only=face)
                        result["i"] = i
                        dat.append(result)

    return dat


## Testing the understanding of the task

In [ ]:
for model in models:
    testOfferCalculation(models)

In [ ]:
for model in models:
    testOfferComparison(models)

In [6]:
# Checking the best possible 
# [0, 195.0, 390.0, 450.0, 525.0, 600] (exp_level =0.1)
# [0, 120.0, 240.0, 600, 600.0, 600] (exp_level =0.3)
# [0, 0.0, 0, 600, 600.0, 600]    (exp_level =1.0)
max_offer = "9-9-9"
min_offer = "1-1-1"
count = 0
temp = []
for i in range(1,10):
    for j in range(1,10):
        for k in range(1,10):
            offer = f"{i}-{j}-{k}"
            response = agentResponse(offer, threshold=0.65, exp_level=0.25)
            if response.startswith('"I think this offer could work.'):
                count +=1
                temp.append( calPayoff([int(n) for n in offer.split("-")]) )
                if calPayoff([int(n) for n in max_offer.split("-")]) < calPayoff([int(n) for n in offer.split("-")]):
                    max_offer = offer
                if calPayoff([int(n) for n in min_offer.split("-")]) > calPayoff([int(n) for n in offer.split("-")]):
                    min_offer = offer
print(count)
print(max_offer, calPayoff([int(n) for n in max_offer.split("-")]))
print(min_offer)
temp.sort()
print(temp)
            

192
1-9-4 550
9-9-9
[0, 15, 30, 30, 45, 45, 50, 60, 60, 60, 65, 75, 75, 75, 80, 80, 90, 90, 90, 95, 95, 100, 105, 105, 110, 110, 110, 115, 120, 120, 120, 125, 125, 125, 130, 130, 135, 135, 140, 140, 140, 145, 145, 150, 150, 150, 155, 155, 160, 160, 160, 165, 165, 170, 170, 175, 175, 180, 180, 180, 180, 185, 185, 190, 190, 190, 195, 195, 195, 200, 200, 200, 205, 205, 210, 210, 210, 210, 215, 215, 220, 220, 225, 225, 230, 230, 230, 235, 240, 240, 240, 245, 245, 245, 250, 250, 250, 255, 255, 260, 260, 260, 260, 265, 265, 270, 270, 275, 275, 280, 280, 280, 285, 290, 290, 290, 295, 295, 295, 300, 300, 300, 305, 305, 310, 310, 310, 310, 315, 315, 320, 320, 325, 325, 330, 330, 330, 335, 340, 340, 345, 345, 350, 350, 355, 360, 360, 360, 365, 365, 370, 370, 375, 375, 380, 380, 380, 385, 390, 390, 395, 395, 400, 400, 405, 410, 410, 415, 420, 425, 430, 430, 430, 435, 440, 440, 445, 445, 450, 455, 460, 460, 470, 475, 480, 485, 490, 490, 500, 505, 520, 550]


## Main Experiments

In [ ]:
#negotiation(OPUS)

In [ ]:
m_to_run = CHATGPT

temp_output = experiment(m_to_run, temperature=0.5, repeat=20)
temp_output_pd = pd.DataFrame(temp_output)
temp_output_pd.to_csv(m_to_run+'_results_updated.csv',index=False)

In [ ]:
m_to_run = FURBO

temp_output = experiment(m_to_run, temperature=0.5, repeat=10)
temp_output_pd = pd.DataFrame(temp_output)
temp_output_pd.to_csv(m_to_run+'_results_0_9_updated.csv',index=False)

temp_output = experiment(m_to_run, temperature=0.5, repeat=10)
temp_output_pd = pd.DataFrame(temp_output)
temp_output_pd.to_csv(m_to_run+'_results_10_19_updated.csv',index=False)

In [ ]:
m_to_run = SONNET

#temp_output = experiment(m_to_run, temperature=0.5, repeat=4)
#temp_output_pd = pd.DataFrame(temp_output)
#temp_output_pd.to_csv(m_to_run+'_results_0_3.csv',index=False)

#temp_output = experiment(m_to_run, temperature=0.5, repeat=4)
#temp_output_pd = pd.DataFrame(temp_output)
#temp_output_pd.to_csv(m_to_run+'_results_4_7.csv',index=False)

#temp_output = experiment(m_to_run, temperature=0.5, repeat=4)
#temp_output_pd = pd.DataFrame(temp_output)
#temp_output_pd.to_csv(m_to_run+'_results_8_11.csv',index=False)

# temp_output = experiment(m_to_run, temperature=0.5, repeat=4)
# temp_output_pd = pd.DataFrame(temp_output)
# temp_output_pd.to_csv(m_to_run+'_results_12_15.csv',index=False)

temp_output = experiment(m_to_run, temperature=0.5, repeat=4)
temp_output_pd = pd.DataFrame(temp_output)
temp_output_pd.to_csv(m_to_run+'_results_16_19.csv',index=False)

In [ ]:
m_to_run = OPUS

# temp_output = experiment(m_to_run, temperature=0.5, repeat=2)
# temp_output_pd = pd.DataFrame(temp_output)
# temp_output_pd.to_csv(m_to_run+'_results_0_1.csv',index=False)

# temp_output = experiment(m_to_run, temperature=0.5, repeat=2)
# temp_output_pd = pd.DataFrame(temp_output)
# temp_output_pd.to_csv(m_to_run+'_results_2_3.csv',index=False)

# temp_output = experiment(m_to_run, temperature=0.5, repeat=2)
# temp_output_pd = pd.DataFrame(temp_output)
# temp_output_pd.to_csv(m_to_run+'_results_4_5.csv',index=False)

# temp_output = experiment(m_to_run, temperature=0.5, repeat=2)
# temp_output_pd = pd.DataFrame(temp_output)
# temp_output_pd.to_csv(m_to_run+'_results_6_7.csv',index=False)

# temp_output = experiment(m_to_run, temperature=0.5, repeat=2)
# temp_output_pd = pd.DataFrame(temp_output)
# temp_output_pd.to_csv(m_to_run+'_results_8_9.csv',index=False)

# temp_output = experiment(m_to_run, temperature=0.5, repeat=2)
# temp_output_pd = pd.DataFrame(temp_output)
# temp_output_pd.to_csv(m_to_run+'_results_10_11.csv',index=False)

# temp_output = experiment(m_to_run, temperature=0.5, repeat=2)
# temp_output_pd = pd.DataFrame(temp_output)
# temp_output_pd.to_csv(m_to_run+'_results_12_13.csv',index=False)

# temp_output = experiment(m_to_run, temperature=0.5, repeat=2)
# temp_output_pd = pd.DataFrame(temp_output)
# temp_output_pd.to_csv(m_to_run+'_results_14_15.csv',index=False)

# temp_output = experiment(m_to_run, temperature=0.5, repeat=2)
# temp_output_pd = pd.DataFrame(temp_output)
# temp_output_pd.to_csv(m_to_run+'_results_16_17.csv',index=False)

# temp_output = experiment(m_to_run, temperature=0.5, repeat=2)
# temp_output_pd = pd.DataFrame(temp_output)
# temp_output_pd.to_csv(m_to_run+'_results_18_19.csv',index=False)